In [1]:
import numpy as np
import pandas as pd

URL_DATA = 'https://storage.googleapis.com/dosm-public-pricecatcher/lookup_premise.parquet'
df_premise_code = pd.read_parquet(URL_DATA)
if 'date' in df_premise_code.columns: df_premise_code['date'] = pd.to_datetime(df_premise_code['date'])


URL_DATA = 'https://storage.googleapis.com/dosm-public-pricecatcher/lookup_item.parquet'
df_item_code = pd.read_parquet(URL_DATA)
if 'date' in df_item_code.columns: df_item_code['date'] = pd.to_datetime(df_item_code['date'])


In [6]:
import numpy as np
import pandas as pd

#!pip install pandas fastparquet

URL_DATA = 'https://storage.googleapis.com/dosm-public-pricecatcher/pricecatcher_2023-07.parquet'

df = pd.read_parquet(URL_DATA)
if 'date' in df.columns: df['date'] = pd.to_datetime(df['date'])
df.head()


,date,premise_code,item_code,price
0,2023-07-01,2,1,9.4
1,2023-07-01,2,9,36.0
2,2023-07-01,2,16,6.0
3,2023-07-01,2,18,6.0
4,2023-07-01,2,19,6.0


In [7]:
def merge_data(df):

    tempdf = df.copy() #must make a copy . doing tempdf=df7 is just reassigning

    print(tempdf.head())
    #
    p =tempdf.pop('premise_code').map(df_premise_code.set_index('premise_code')["premise_type"])
    p.name = 'premise_type'


    tempdf = df.copy() #must make a copy . doing tempdf=df7 is just reassigning
    q =tempdf.pop('premise_code').map(df_premise_code.set_index('premise_code')["premise"])
    q.name = 'premise'

    tempdf = df.copy() #must make a copy . doing tempdf=df7 is just reassigning
    s =tempdf.pop('premise_code').map(df_premise_code.set_index('premise_code')["state"])
    s.name = 'state'

    tempdf = df.copy() #must make a copy . doing tempdf=df7 is just reassigning
    t = tempdf.pop('premise_code').map(df_premise_code.set_index('premise_code')['district'])
    t.name = "district"

    tempdf = df.copy() #must make a copy . doing tempdf=df7 is just reassigning
    u = tempdf.pop('item_code').map(df_item_code.set_index('item_code')['item'])
    u.name = "item"

    tempdf = df.copy() #must make a copy . doing tempdf=df7 is just reassigning
    v = tempdf.pop('item_code').map(df_item_code.set_index('item_code')['unit'])
    v.name = "unit"

    tempdf = df.copy() #must make a copy . doing tempdf=df7 is just reassigning
    w = tempdf.pop('item_code').map(df_item_code.set_index('item_code')['item_category'])
    w.name = "item_category"

    #print('S:\n',s) 
    #print('T:\n',t)

    df_master = pd.concat([df,p,q,s,t,u,v,w],axis=1)
    return df_master    

In [8]:

df_master = merge_data(df)    
#df_master.sample(frac=1)
df_master.head()

        date  premise_code  item_code  price
0 2023-07-01             2          1    9.4
1 2023-07-01             2          9   36.0
2 2023-07-01             2         16    6.0
3 2023-07-01             2         18    6.0
4 2023-07-01             2         19    6.0


,date,premise_code,item_code,price,premise_type,premise,state,district,item,unit,item_category
0,2023-07-01,2,1,9.4,Pasar Basah,PASAR BESAR IPOH,Perak,Kinta,AYAM BERSIH - STANDARD,1kg,AYAM
1,2023-07-01,2,9,36.0,Pasar Basah,PASAR BESAR IPOH,Perak,Kinta,DAGING KAMBING BEBIRI IMPORT BERTULANG (MUTTON...,1kg,DAGING
2,2023-07-01,2,16,6.0,Pasar Basah,PASAR BESAR IPOH,Perak,Kinta,BETIK BIASA,1kg,BUAH-BUAHAN
3,2023-07-01,2,18,6.0,Pasar Basah,PASAR BESAR IPOH,Perak,Kinta,PISANG BERANGAN,1kg,BUAH-BUAHAN
4,2023-07-01,2,19,6.0,Pasar Basah,PASAR BESAR IPOH,Perak,Kinta,PISANG EMAS,1kg,BUAH-BUAHAN


In [25]:
df_by_item_state = df_master.groupby(['item','state']).price.agg(['max','min','mean'])
df_by_item_state.head(20)




max    min       mean
item                 state                                     
100 PLUS (ORIGINAL)  Johor               4.70   3.19   3.755636
                     Kedah               4.50   3.19   3.696923
                     Kelantan            4.60   3.19   3.963500
                     Melaka              3.99   2.99   3.578182
                     Negeri Sembilan     4.00   2.99   3.531053
                     Pahang              4.20   2.99   3.613611
                     Perak               4.20   2.99   3.616167
                     Perlis              4.01   3.49   3.789091
                     Pulau Pinang        4.50   3.10   3.743235
                     Sabah               4.50   3.30   3.986049
                     Sarawak             5.00   3.35   3.973153
                     Selangor            4.50   2.99   3.665179
                     Terengganu          4.50   3.29   3.811875
                     W.P. Kuala Lumpur   4.70   3.19   3.849136
                     W.P. Labuan         4.20   3.70   3.940000
                     W.P. Putrajaya      4.50   3.19   3.989286
ANGGUR HIJAU BERBIJI Johor               6.99   6.99   6.990000
                     Kelantan           16.90  16.90  16.900000
                     Pahang             13.99   9.99  11.990000
                     Perlis              7.00   7.00   7.000000